# RAG关于抗炎性肽序列的批量判断

In [1]:
import pandas as pd
#载入预处理后的patent_seq数据
df_seq=pd.read_csv('./patent_seq_list_new.csv')

## Llama2相关组件初始化（本地llama）

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from IPython.display import Markdown, display

In [3]:
import torch
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate

import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

LLAMA2_7B_chat = "/home/cymei/llama_hf/llama-2-chat-7b-hf"
LLAMA2_13B_chat = "/mnt/sdc/Llama-2-13b-chat-hf"
LLAMA2_7B_hf = "/home/cymei/llama_hf/llama-2-7b-hf"

selected_model = LLAMA2_7B_chat #LLAMA2_7B_chat

SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True},
)

/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:107: UserWarning: 

BNB_CUDA_VERSION=XXX can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64
Loading CUDA version: BNB_CUDA_VERSION=118


  warn((f'\n\n{"="*80}\n'


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 商用大模型API加载

In [2]:
import os

os.environ["http_proxy"] = "http://192.168.1.254:7890"
os.environ["https_proxy"] = "http://192.168.1.254:7890"

In [3]:
from llama_index.llms.perplexity import Perplexity

pplx_api_key = "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"

llm = Perplexity(
    api_key=pplx_api_key, model="mixtral-8x7b-instruct", temperature=0.0,
    api_base="https://api.perplexity.ai",
)

In [4]:
from llama_index import SimpleDirectoryReader
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context,
)

from llama_index.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)

from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage import StorageContext
from llama_index.node_parser import get_leaf_nodes, get_root_nodes
from llama_index import VectorStoreIndex
# from llama_index.llms import LLM
from llama_index.retrievers.auto_merging_retriever import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

#载入模型以及embedding功能
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(
    model_name="/home/cymei/bge-small-en",device='cuda:2'
)

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model #"local:/home/cymei/bge-small-en"
)
set_global_service_context(service_context)


## 混合索引引擎

In [5]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
)
from llama_index.schema import BaseNode

from typing import List

In [6]:
from nltk import word_tokenize 

class CustomRetriever(BaseRetriever):
    """自定义的混合索引类：相似度索引+关键词索引"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        vector_retriever_large: VectorIndexRetriever,
        max_num_keyword_nodes=3,
        keywords=[]
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever #头部相似度节点
        self._vector_retriever_large = vector_retriever_large #更大范围的头部相似度节点(用于关键词搜索)

        self.keywords=keywords #索引依据的关键词
        self.max_num_keyword_nodes=max_num_keyword_nodes #设置最大关键词节点数量
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        #利用两个不同的参数的retriever进行retrieve
        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        vector_nodes_large = self._vector_retriever_large.retrieve(query_bundle)

        #确保集合中的节点id对应的相似度由大到小排列
        vector_ids = {n.node_id for n in sorted(vector_nodes,key=lambda node: node.score,reverse=True)} 
        # vector_ids_large = {n.node_id for n in vector_nodes_large}

        #对于更大范围的相似度索引结果，取出其中含有关键词的节点
        keyword_ids = []
        for n in sorted(vector_nodes_large,key=lambda node: node.score,reverse=True):
            for k in self.keywords:
                if(k in word_tokenize(n.get_content())):
                    #判断关键词是否在文章片段的分词结果中
                    keyword_ids.append(n.node_id)
                    break

        combined_dict = {n.node_id: n for n in vector_nodes}
        combined_dict.update({n.node_id: n for n in vector_nodes_large if n.node_id in keyword_ids })

        #合并两组节点
        if(keyword_ids==[]):
            #不含有关键词的情况下照常进行相似度索引
            retrieve_ids = vector_ids
        else:
            keyword_ids_top=set(keyword_ids[:self.max_num_keyword_nodes]) #取相似度最高的几个关键词节点
            vector_ids_unique=vector_ids-keyword_ids_top  #top相似度集合中独有的节点
            retrieve_ids=keyword_ids_top #关键词集合和top相似度集合共有的节点+关键词集合中独有的节点
            add_num=len(vector_ids)-len(keyword_ids_top)
            retrieve_ids=set(list(vector_ids_unique)[:add_num]).union(retrieve_ids) #额外添加部分top相似度集合中独有的节点

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [7]:
#创建RAG工作流的类
class LLM_summary:
    def __init__(self,documents=[],keywords=[]):
        #用于检索的关键词
        self.keywords=keywords
        #载入专利文本
        self.documents=documents
    
    def hierarchy_node_index(self,chunk_list=[1024, 512, 256],similarity_top_k=6,max_num_keyword_nodes=3):
        '''
        获取分层节点，并建立对应索引
        设置每一层的token数量、选入prompt的节点数量以及最多含有关键词的节点数量
        '''
        node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_list)
        #获取分段后的每个节点
        self.nodes = node_parser.get_nodes_from_documents(self.documents)

        #建立各个节点的存储
        docstore = SimpleDocumentStore()
        # insert nodes into docstore
        docstore.add_documents(self.nodes)
        # define storage context (will include vector store by default too)
        storage_context = StorageContext.from_defaults(docstore=docstore)
        self.leaf_nodes = get_leaf_nodes(self.nodes)
    
        # 利用叶子节点计算相似度，并关联到对应的父节点
        base_index = VectorStoreIndex(
            self.leaf_nodes,
            storage_context=storage_context,
        )

        #用于寻找和问题最相近的头部文本片段
        base_retriever = base_index.as_retriever(similarity_top_k=similarity_top_k)
        #用于从更大范围中寻找含有关键词的文本片段
        large_retriever = base_index.as_retriever(similarity_top_k=len(self.leaf_nodes))
        #构建混合索引
        custom_retriever = CustomRetriever(base_retriever, large_retriever,max_num_keyword_nodes=max_num_keyword_nodes,keywords=self.keywords)

        retriever = AutoMergingRetriever(custom_retriever, storage_context, verbose=True)

        return retriever 

## 批量seq判断

In [8]:
from llama_index.llms import ChatMessage

#设置system_prompt(针对API的模型)
system_str=  """You are a biologist and patent expert. 
    You will be provided with some contents from a patent and will be asked to answer specific questions related to the patent. 
    Please answer the question only using the provided contents and do not make up the answer with prior knowledge."""

In [9]:
import json
import os
import re
from llama_index.schema import Document
os.chdir('/home/cymei/nlp_task/anti_inflammation')

#专利文本载入函数
#获取claim和description的信息并创建对应的document实例
def load_doc(file_name):
    with open(file_name,'r') as f:
        js_list=json.load(f)
        for js in js_list:
            if(js['article_type']=='claim'):
                claims_raw='The claim information:\n'+js['text']
            elif(js['article_type']=='description'):
                description_raw='The description information:\n'+js['text']
    
    #文本预处理
    claims1=re.sub('\u2003',' ',claims_raw)
    claims_real=re.sub('SEQ ID N[oO][.: ]+','SEQ_ID_NO_',claims1)
    description1=re.sub('\u2003',' ',description_raw)
    description_real=re.sub('SEQ ID N[oO][.: ]+','SEQ_ID_NO_',description1)

    doc_claim = Document(text=claims_real)
    doc_des = Document(text=description_real)

    return {'claim':doc_claim,'description':doc_des}

In [10]:
import os
from IPython.display import Markdown, display
import time

os.environ["http_proxy"] = "http://192.168.1.254:7890"
os.environ["https_proxy"] = "http://192.168.1.254:7890"

df_seq['is_anti_inflammation']=0
df_seq['model_response']=''
df_seq['supportive_context']=''

patent_list=df_seq['patent_no'].unique() 

#对每一个patnet下的seq进行判断
for patent in patent_list:

    df_unit=df_seq[df_seq['patent_no']==patent]
    file_loc='./patents_parse/{}.json'.format(patent)
    doc_dic=load_doc(file_loc)
    
    #使用llama-index建立查询引擎
    #分别从claim和des中进行检索(防止因为相似度的问题全部使用description中的信息从而遗漏claim的信息)
    summarier_claim=LLM_summary([doc_dic['claim']])
    summarier_des=LLM_summary([doc_dic['description']])

    for index,row in df_unit.iterrows():

        summarier_claim.keywords=[row['seq_no']]
        summarier_des.keywords=[row['seq_no']]

        retriever_claim=summarier_claim.hierarchy_node_index(similarity_top_k=2,max_num_keyword_nodes=2)
        retriever_des=summarier_des.hierarchy_node_index(similarity_top_k=8,max_num_keyword_nodes=3)

        seq_peptides='{}:{}'.format(row['seq_no'],row['seq_peptides'])
        question="""Given content describing the functions and applications of the patent.
        Please determine if sequence number "{0}" have the ability of anti-inflammation. 
        You should focus on sequence number "{0}" and ignore other irrelvant sequence number!
        Finally, plesae do analysis and explain the reason first, then print "The answer is: 'Yes'" or "The answer is: 'No'" in next line.
        Don't answer like "Possible" or "Maybe"! Please return the top 3 most supportive context number.
            """.format(seq_peptides)

        #分别从claim和des中进行检索，确保两部分内容都会被用上
        response_nodes_claim=retriever_claim.retrieve(question) 
        response_nodes_des=retriever_des.retrieve(question) 
        response_nodes=response_nodes_claim + response_nodes_des
        #print(question)
        context_str="""Context is :
            {}
            ---------------------
            Given the provided context, please answer the query:
            {}
            """.format('\n'.join(['Context{}:\n'.format(index) + node.text for index,node in enumerate(response_nodes)]),question)  
        messages_dict = [
            {"role": "system", "content": system_str},
            {"role": "user", "content": context_str},
        ]
        messages = [ChatMessage(**msg) for msg in messages_dict]
        response = llm.chat(messages)
        #print(response.raw['usage'])
        display(Markdown(f"<b>{response.raw['usage']}</b>"))
        print('--------------------------------------------------------------------')
        # display(Markdown(f"<b>{response}</b>"))

        response_ans=response.message.content
        context_list=re.findall('Context[0-9]+',response_ans)
        context_nums=set([int(re.sub('Context','',context)) for context in context_list])
        #目标seq
        condition=(df_seq['patent_no']==patent) & (df_seq['seq_no']==row['seq_no'])
        if('Yes' in response_ans):
            df_seq.loc[condition,'is_anti_inflammation']=1
        elif('No' in response_ans):
            df_seq.loc[condition,'is_anti_inflammation']=0
        else:
            #没有明确yes或no的seq认为是possible
            df_seq.loc[condition,'is_anti_inflammation']=0.5

        df_seq.loc[condition,'model_response']=response_ans
        df_seq.loc[condition,'supportive_context']='\n'.join(['Context{}:\n'.format(index) + response_nodes[index].text for index in context_nums if(index<len(response_nodes))])
        time.sleep(30) #设置间隔防止api调用频率过高(主要由于perplexity对10min内接收的token数量有限制)

> Merging 2 nodes into parent node.
> Parent node id: 7456460c-7716-4da3-980d-6ba0a152d0c2.
> Parent node text: The claim information:
A peptide comprising the amino acid sequence KMIKP, comprising less than 7...

> Merging 1 nodes into parent node.
> Parent node id: 1e66675d-838f-48c5-915d-57244233e85a.
> Parent node text: The claim information:
A peptide comprising the amino acid sequence KMIKP, comprising less than 7...

> Merging 2 nodes into parent node.
> Parent node id: 22dc3451-7ce5-4861-a0ec-be7e436aaf38.
> Parent node text: The description information:
The present invention particularly relates to a method for improving...

> Merging 3 nodes into parent node.
> Parent node id: 9d3beab0-b2e6-4958-8e58-b17179d388f3.
> Parent node text: The conclusion to be drawn is that topically applied peptide 3 inhibits one or more biological pr...



<b>{'prompt_tokens': 2679, 'completion_tokens': 166, 'total_tokens': 2845}</b>

--------------------------------------------------------------------
> Merging 2 nodes into parent node.
> Parent node id: dc21581e-b34b-4172-b669-cdd31eedc6a4.
> Parent node text: The claim information:
A peptide comprising the amino acid sequence KMIKP, comprising less than 7...

> Merging 1 nodes into parent node.
> Parent node id: 4ec6b3ab-ea39-47ba-839c-dc56a2a01b1d.
> Parent node text: The claim information:
A peptide comprising the amino acid sequence KMIKP, comprising less than 7...

> Merging 2 nodes into parent node.
> Parent node id: 220d3152-3d04-42dc-82c0-9aa5a5f4d620.
> Parent node text: Preferably, the inflammatory skin condition is selected from the group consisting of psoriasis, l...

> Merging 2 nodes into parent node.
> Parent node id: 851b2629-0979-4108-9092-d91d76a1baac.
> Parent node text: The conclusion to be drawn is that topically applied peptide 3 inhibits one or more biological pr...

> Merging 2 nodes into parent node.
> Parent node id: 2db040cb-55ce-4086-b3

<b>{'prompt_tokens': 2967, 'completion_tokens': 235, 'total_tokens': 3202}</b>

--------------------------------------------------------------------
> Merging 2 nodes into parent node.
> Parent node id: 69d7c6e5-1a08-4164-a702-f4e877020bd9.
> Parent node text: The claim information:
A peptide comprising the amino acid sequence KMIKP, comprising less than 7...

> Merging 1 nodes into parent node.
> Parent node id: e5cd730a-9aae-475e-b1cf-c547a390229f.
> Parent node text: The claim information:
A peptide comprising the amino acid sequence KMIKP, comprising less than 7...

> Merging 2 nodes into parent node.
> Parent node id: 3fe6a4a4-b49d-4c20-a25c-696cb537f55d.
> Parent node text: The description information:
The present invention particularly relates to a method for improving...

> Merging 3 nodes into parent node.
> Parent node id: 59438054-cb6d-4932-8be8-8bcfde1616d2.
> Parent node text: The conclusion to be drawn is that topically applied peptide 3 inhibits one or more biological pr...



...

--------------------------------------------------------------------


<b>{'prompt_tokens': 2901, 'completion_tokens': 306, 'total_tokens': 3207}</b>

--------------------------------------------------------------------


<b>{'prompt_tokens': 2881, 'completion_tokens': 266, 'total_tokens': 3147}</b>

--------------------------------------------------------------------


In [11]:
#结果存储
df_seq.to_csv('anti_inflammation_res.csv',index=False)  

In [25]:
#按patent统计符合要求的seq数量情况
df_seq[['patent_no','is_anti_inflammation']].groupby(['patent_no']).agg({'is_anti_inflammation':'sum','patent_no':'count'})

,is_anti_inflammation,patent_no
patent_no,,
JP2009518370A,6.0,18
JP2016084331A,1.0,1
KR20110122548A,5.5,6
KR20150042447A,5.0,8
KR20150050627A,1.0,1
KR20150050631A,1.0,1
KR20160003611A,3.0,3
KR20160147293A,1.0,1
KR20170027619A,1.5,3


In [11]:
response.message.content

'After analyzing the provided contexts, the answer to the question is: "The answer is: No". The reason for this answer is that there is no direct mention or evidence in the given contexts that SEQ_ID_NO_1:GKCSTRGRKCCRRKK has the ability to reduce inflammation. Context0 states that SEQ_ID_NO_2 is the sequence where cysteine is replaced with methionine to prevent oxidation in the SS bond and to have an ability to reduce inflammation and promote differentiation from stem cells to chondrocytes. However, it does not mention anything about SEQ_ID_NO_1 having anti-inflammatory properties. Context1, Context2, Context3, and Context5 discuss the inflammation-reducing ability of the peptides, but they do not specifically mention SEQ_ID_NO_1. Therefore, based on the given contexts, it is concluded that SEQ_ID_NO_1:GKCSTRGRKCCRRKK does not have the ability of anti-inflammation.'

In [21]:
#输入prompt中的节点
#做embedding相似度匹配的只有query中的内容，并不涉及整个prompt
from llama_index.response.notebook_utils import display_source_node
for node in response_nodes:
    display_source_node(node, source_length=20000)
    print("--------")

**Node ID:** f27e5a4c-a7cf-4afe-bb9c-9b4b2cd1a05a<br>**Similarity:** 0.8680149987093815<br>**Text:** The claim information:
One antimicrobial peptide selected from the group of peptides having the amino acid sequence shown in SEQ_ID_NO_1 to 7. A pharmaceutical composition for antimicrobial use comprising the peptide of claim 1 as an active ingredient. [3] The composition according to claim 2, wherein the composition is a Gram positive bacterium and has antibacterial activity against any strain selected from the group consisting of B. subtilis , S. aureus , S. epidermidis , Gram-negative bacteria, E. coli , P. aeruginosa , and S. typhimurium &Lt; / RTI &gt; 3. The pharmaceutical composition according to claim 2, wherein the effective amount of the peptide is 20 uM at 1 uM. The composition according to claim 2, wherein the composition has no toxicity to any cell line selected from the group consisting of human red blood cells and epithelial cells. 1 to 7, acting on the cell membrane of a microorganism selected from the group consisting of Gram-positive bacteria, Gram-negative bacteria, multidrug-resistant bacteria and fungi. A pharmaceutical composition for antiinflammation comprising one peptide selected from the group consisting of peptides having the amino acid sequence represented by SEQ ID NOS: 1 to 7 as an active ingredient. 1. A cosmetic composition comprising, as an active ingredient, one peptide selected from the group consisting of peptides having an amino acid sequence represented by SEQ ID NOS: 1 to 7.<br>

--------


**Node ID:** b5b804c1-e23e-49be-bf83-84fc1c1e8df4<br>**Similarity:** 0.896793670562688<br>**Text:** The cytotoxicity of the peptides was measured as a percentage of the absorbance value of the wells treated with the peptide divided by the absorbance value of the control cells not treated with the peptide. As shown in FIG. 2, 9Pbw3, which is a comparative peptide, and the peptides of SEQ ID NOS: 1 to 7, which are antibiotic peptides, showed a survival rate of 100% at a concentration having an activity, and the toxicity to human epidermal cells was very small. Example 7: Raw264 .7 cell in Pathogen LPS NO (&lt; / RTI &gt; produced by stimulation nitrite oxide production In order to examine whether the peptides represented by SEQ. ID. Nos. 1 to 7 are effective for antiinflammatory activity, the following procedure was carried out.<br>

--------


**Node ID:** b86ce809-b35e-4356-bda1-93a157c7c6f5<br>**Similarity:** 0.8940963121214376<br>**Text:** The antibiotic peptide of SEQ_ID_NO_5 lowered the hydrophobicity of 9Pbw3 of the comparative peptide, The positively charged arginine is shifted to the N-terminal side By this it was confirmed that the effective charge seen by SEQ_ID_NO_4 was then excellent antibacterial activity and anti-inflammatory activity with respect to all the strains designed to positively charged antimicrobial peptide sex than other derivatives. For this reason, the antibiotic peptides of SEQ ID NOS: 4 and 5 above are the peptides that best meet the objectives of the present invention to lower the cytotoxicity while retaining their activity as compared to the comparative peptide 9Pbw3, and are also useful as safe antibiotics and anti- It will be useful. Hereinafter, the present invention will be described more specifically by way of examples. It should be apparent to those skilled in the art that these embodiments are illustrative only of the present invention and that the scope of the present invention is not limited by these embodiments. It will be apparent to those skilled in the art that a pharmaceutical composition for an antimicrobial agent containing an antimicrobial peptide derivative according to the present invention having an antibacterial and antifungal activity as an active ingredient may be used although it is not shown in a specific example. Peptides Amino acid sequence Hydrophobicity SEQ_ID_NO_1: 9Pbw3-1 RLWLAIFRR-NH 2 1.85 SEQ_ID_NO_2: 9Pbw3-2 RLWAAIWRR-NH 2 0.62 SEQ_ID_NO_3: 9Pbw3-3 RLWLAAWRR-NH 2 0.73 SEQ_ID_NO_4: 9Pbw3-4 RLWLAILRR-NH 2 1.82 SEQ_ID_NO_5: 9Pbw3-5 RRWLAIWRR-NH 2 -0.36 SEQ_ID_NO_6: 9Pbw3-6 RLFLAIWRR-NH 2 1.85 SEQ_ID_NO_7: 9Pbw3-7 RLLLAIWRR-NH 2 1.82 Comparative peptide SEQ_ID_NO_8: 9 Pbw3 RLWLAIWRR-NH2 1.82 Table 1 shows comparison peptides of SEQ ID NOS: 1 to 7 and amino acid sequence of SEQ.<br>

...

--------


**Node ID:** 111ed181-e49b-4843-ad31-b11ee164b043<br>**Similarity:** 0.8930668300788075<br>**Text:** The antimicrobial peptides labeled with SEQ ID NOS: 1, 4, 6 and 7, which show similar hydrophobicity to the comparative peptide 9Pbw3, and SEQ_ID_NO_5 with one more positive charge of the effective charge compare the nitrite values of the control without LPS treatment The production of NO (nitric oxide) was suppressed to a level similar to that of the comparative peptide 9Pbw3. From the above results, 9Pbw3-4 of SEQ_ID_NO_4 among the antibiotic peptides of SEQ ID NOS: 1 to 7 of the present invention exhibited the most excellent antibacterial and antifungal activity among the seven designed antibiotic peptides, exhibited excellent anti-inflammatory activity, , Which is an antibiotic peptide that increases cell selectivity due to low hemolytic activity and cytotoxicity compared to 9Pbw3. It was found that the presence of hydrophobic residues at the residue position is important for the activity and that substitution with Leu residues rather than Trp is helpful in reducing the toxicity.<br>

--------


**Node ID:** 9fe1f041-02cd-4aaa-af53-48ff7979b3e1<br>**Similarity:** 0.8912240330433616<br>**Text:** In one embodiment of the present invention, the anti-inflammatory activity is characterized by inhibiting the production of an inducer nitrate oxide. In one embodiment of the present invention, the anti-inflammatory composition is preferably not toxic to human red blood cells or normal mammalian cells. Hereinafter, the present invention will be described. In the present invention, an antimicrobial peptide isolated from a larva of a white spotted flower, a high antimicrobial and anti-inflammatory peptide derivative designed from a residue sequence of a proteasomicin are synthesized to exhibit antimicrobial activity and anti-inflammatory activity. Hereinafter, the present invention will be described in more detail with reference to the steps. In the present invention, based on the amino acid sequence of the protease-antimicrobial peptide helix domain, it has a high antibiotic activity, and based on the 9Pbw3 residue sequence of the patent (Patent Registration No.<br>

--------
